<a href="https://colab.research.google.com/github/kjw3757/Guide-Book/blob/main/bard%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%B1%85_%EB%82%9C%EC%9D%B4%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Python Package BardAPI tutorial

## Install

In [ ]:
!pip install bardapi>=0.1.8
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00


## **BARD 이용해서 엑셀 돌리기**

In [ ]:
import os
import bardapi
import openpyxl
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from bardapi import Bard

token='XAhVqxRYtzCF-CC0DElTeLBtCtqQeLdhiTRIGJmdqt2hqFWfBFK2TSs1AkbYfUCAMPdTGA.'


In [ ]:
# 엑셀 파일 로딩
workbook = openpyxl.load_workbook("/content/book_data_hana_test (2).xlsx")
sheet = workbook.active# 필요한 열 추출
data = pd.DataFrame(sheet.values)

book_data = data.iloc[1:, [0, 1, 2, 3]]
book_data.columns = ["title", "author", "category", "publisher"]

In [ ]:
def get_book_difficulty(title, author, category, publisher):
    if title is None or author is None or category is None or publisher is None:
        return "Not available"

    # Set up the book information for the prompt
    book_info = f"Book: {title} by {author}, Category: {category}, Publisher: {publisher}"
    prompt_with_book_info = (
        "This Excel file contains the following information: Title of the book, Publisher, Author, Category.\n"
        f"Read each row from this Excel file and use the information to determine what kind of book it is and assign a difficulty rating from 1 to 5. "
        f"The rating should be determined by considering various factors such as the length of the book, complexity of the content, language level, and subject matter. \n"
        f"For each book, output the title, author and the difficulty rating, and summarize the reason why you determine the difficulty like that.\n"
        + book_info
    )

    # Generate text from the Bard API
    response = bardapi.core.Bard(token).get_answer(prompt_with_book_info)

    # Extract the difficulty rating from the generated text
    difficulty = response['content'].strip()

    return difficulty


In [ ]:
# 새로운 엑셀 파일 생성
new_workbook = openpyxl.Workbook()
new_sheet = new_workbook.active
new_sheet.append(["Title", "Author", "Category", "Publisher", "Difficulty", "Content"])


# **책 소개**

In [ ]:
def get_book_content(title, author, category, publisher):
    if title is None or author is None or category is None or publisher is None:
        return "Not available"

    # Set up the book information for the prompt
    book_info = f"Book: {title} by {author}, Category: {category}, Publisher: {publisher}"
    prompt_with_book_info = (
       "This Excel file contains the following information: book titles, publishers, authors, categories.\n"
        f"Please read each line from this excel file and use the information to explain the contents and outline of each book using English."
        + book_info
    )

    # Generate text from the Bard API
    response = bardapi.core.Bard(token).get_answer(prompt_with_book_info)

    # Extract the difficulty rating from the generated text
    overview = response['content'].strip()

    return overview

In [ ]:
# 모든 책에 대해 개요 출력
for i, row in book_data.iterrows():
    title, author, category, publisher = row["title"], row["author"], row["category"], row["publisher"]
    difficulty = get_book_difficulty(title, author, category, publisher)
    content = get_book_content(title, author, category, publisher)
    new_sheet.append([title, author, publisher , category, difficulty, content])

# **엑셀 저장**

In [ ]:
# 새로운 엑셀 파일 저장
new_workbook.save("test-with-difficulty.xlsx")

# 난이도 추출

In [ ]:
# 엑셀 파일 경로 설정
excel_file_path = "/content/test-with-difficulty.xlsx"

# 엑셀 파일 읽기
df = pd.read_excel(excel_file_path)

# Difficulty 열의 값을 수정
df['Difficulty'] = df['Difficulty'].str.extract(r'Difficulty rating: (\d+)', expand=False)

# 수정된 데이터프레임을 새로운 엑셀 파일로 저장
output_file_path = "test_bard_result.xlsx"
df.to_excel(output_file_path, index=False)